In [1]:
import pandas as pd
import pyreadstat as st
pd.set_option('display.max_columns', None)
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases prácticas\PS\PS3-20251111\Panel101.dta"

df, meta = st.read_dta(path)
df.head(1)

,country,year,y,y_bin,x1,x2,x3,opinion,op,Y,time,treated,did,_diff
0,1,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,1.0,1.0,1.342788,0.0,0.0,0.0,0.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  70 non-null     int64  
 1   year     70 non-null     int64  
 2   y        70 non-null     float64
 3   y_bin    70 non-null     float64
 4   x1       70 non-null     float64
 5   x2       70 non-null     float64
 6   x3       70 non-null     float64
 7   opinion  70 non-null     float64
 8   op       70 non-null     float64
 9   Y        70 non-null     float64
 10  time     70 non-null     float64
 11  treated  70 non-null     float64
 12  did      70 non-null     float64
 13  _diff    70 non-null     float64
dtypes: float64(12), int64(2)
memory usage: 7.8 KB


In [3]:
meta.column_names_to_labels

{'country': 'Country',
 'year': 'Year',
 'y': 'Outcome Y',
 'y_bin': 'Binary outcome Y',
 'x1': 'Predictor x1',
 'x2': 'Predictor x2',
 'x3': 'Predictor x3',
 'opinion': 'Categorical variable',
 'op': None,
 'Y': None,
 'time': None,
 'treated': None,
 'did': None,
 '_diff': 'Diff-in-diff'}

In [5]:
df.describe()

,country,year,y,y_bin,x1,x2,x3,opinion,op,Y,time,treated,did,_diff
count,70.000000,70.00000,7.000000e+01,70.000000,70.000000,70.000000,70.000000,70.000000,70.00000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,4.000000,1994.50000,1.845072e+09,0.800000,0.648001,0.133869,0.761851,2.442857,0.50000,1.845072,0.600000,0.428571,0.257143,0.257143
std,2.014441,2.89302,3.015167e+09,0.402888,0.468070,1.371099,1.446332,1.137448,0.50361,3.015167,0.493435,0.498445,0.440215,0.440215
min,1.000000,1990.00000,-7.863483e+09,0.000000,-0.567575,-1.621761,-1.165387,1.000000,0.00000,-7.863483,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1992.00000,2.465716e+08,1.000000,0.328960,-1.215649,-0.079315,1.000000,0.00000,0.246572,0.000000,0.000000,0.000000,0.000000
50%,4.000000,1994.50000,1.897898e+09,1.000000,0.641262,-0.462143,0.514191,2.500000,0.50000,1.897898,1.000000,0.000000,0.000000,0.000000
75%,6.000000,1997.00000,3.372262e+09,1.000000,1.095843,1.607754,1.154864,3.000000,1.00000,3.372262,1.000000,1.000000,0.750000,0.750000
max,7.000000,1999.00000,8.941232e+09,1.000000,1.446412,2.530277,7.168922,4.000000,1.00000,8.941232,1.000000,1.000000,1.000000,1.000000


In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Parto de tu df original
df_reg = df.copy()

# ==== Generar las variables como en Stata ====

# Y = y / 1e9
df_reg["Y"] = df_reg["y"] / 1_000_000_000

# time = (year >= 1994) & !missing(year)
df_reg["time"] = ((df_reg["year"] >= 1994) & df_reg["year"].notna()).astype(int)

# treated = (country > 4) & !missing(country)
df_reg["treated"] = ((df_reg["country"] > 4) & df_reg["country"].notna()).astype(int)

# did = time * treated
df_reg["did"] = df_reg["time"] * df_reg["treated"]

# ==== Misma muestra que Stata: sin missing en las variables del modelo ====
vars_modelo = ["Y", "time", "treated", "did"]
df_reg = df_reg.dropna(subset=vars_modelo)

# ==== Regresión OLS: Y time treated did ====
X = df_reg[["time", "treated", "did"]]
X = sm.add_constant(X)  # agrega _cons
y = df_reg["Y"]

model = sm.OLS(y, X)
res = model.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.984
Date:                Fri, 14 Nov 2025   Prob (F-statistic):              0.125
Time:                        18:43:26   Log-Likelihood:                -173.06
No. Observations:                  70   AIC:                             354.1
Df Residuals:                      66   BIC:                             363.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3581      0.738      0.485      0.6